In [22]:
import json

# Specify the file name and location
filename = "tiny.json"

# Read the JSON file
with open(filename, 'r') as file:
    data = json.load(file)

# Extract the details from the JSON
# For example, extract all shops
linear_model = True  # Set to False if you don't want a linear model
shops = data["shops"]
print("Shops:")
print(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles:
    if linear_model:
        # Set all vehicles to regular if linear_model is true
        print("Vehicle Type: regular")
    else:
        # Otherwise, print the actual vehicle type
        print(f"Vehicle ID: {vehicle['id']}, Type: {vehicle['type']}")

for constraint in constraints:
    print(f"Constraint: {constraint}")

for parameter in parameters:
    print(f"Parameter: {parameter}")

for shop in shops:
    print(f"Shop: {shop}")


Shops:
[{'name': 'body', 'resequencing_lag': 1}, {'name': 'paint', 'resequencing_lag': 2}, {'name': 'assembly', 'resequencing_lag': 0}]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Constraint: {'id': 1, 'type': 'batch_size', 'shop': 'body', 'cost': 2, 'min_vehicles': 2, 'max_vehicles': 4, 'vehicles': [1, 2, 4]}
Constraint: {'id': 2, 'type': 'lot_change', 'shop': 'paint', 'cost': 3, 'partition': [[1, 2], [3, 4, 5]]}
Constraint: {'id': 3, 'type': 'rolling_window', 'shop': 'assembly', 'cost': 2, 'window_size': 3, 'max_vehicles': 2, 'vehicles': [1, 2, 5]}
Constraint: {'id': 4, 'type': 'batch_size', 'shop': 'assembly', 'cost': 3, 'min_vehicles': 2, 'max_vehicles': 3, 'vehicles': [1, 2, 3]}
Constraint: {'id': 5, 'type': 'batch_size', 'shop': 'assembly', 'cost': 3, 'min_vehicles': 1, 'max_vehicles': 2, 'vehicles': [4, 5]}
Parameter: two_tone_delta
Parameter: resequencing_cost
Shop: {'name': 'body', 'resequencing_lag': 1}
Shop: {

In [23]:
# Extract parameters and shop details
resequencing_cost = data["parameters"]["resequencing_cost"]
number_of_shops = len(shops)

for shop in shops:
    print(f"Shop: {shop}")
    
print(f"Number of shops: {number_of_shops}")

number_of_vehicles = len(vehicles)
print(f"Number of vehicles: {number_of_vehicles}")


Shop: {'name': 'body', 'resequencing_lag': 1}
Shop: {'name': 'paint', 'resequencing_lag': 2}
Shop: {'name': 'assembly', 'resequencing_lag': 0}
Number of shops: 3
Number of vehicles: 5


In [24]:
import gurobipy as grb

# Initialize an empty dictionary to store the mapping
shop_map = {}
lag = {}

# In shop_map, every shop name is a key, and the value is a list of constraints associated with that shop
# In lag, every shop name is a key, and the value is the resequencing lag
for shop in shops:
    shop_map[shop["name"]] = []
    lag[shop["name"]] = shop["resequencing_lag"]

n = number_of_vehicles
print(f"n: {n}")

# Initialize Gurobi model (similar to how it's done in Julia with JuMP)
# model = grb.Model("example_model")

# Iterate through constraints and associate them with the corresponding shop in shop_map
for constraint in constraints:
    name = constraint["shop"]
    if name in shop_map:
        shop_map[name].append(constraint)
    else:
        print(f"Warning: Shop name {name} in constraints does not exist in shops.")


n: 5


In [37]:
def permutation(x,n) :
    permutation = [0 for i in range(n)]
    for i in range(n):
        for j in range(n) :
            if x[i+n*j] == 1 :
                permutation[i] = j + 1

    return permutation

In [38]:
import gurobipy as grb

def solve_shop_map(shop_map, n):
    L= []
    for shop, constraints in shop_map.items():
        print(f"Shop: {shop}")
        
        # Create a Gurobi model
        model = grb.Model("Shop_Optimization")
        
        # Decision variable: x[v, t]
        x = model.addVars(range(1, n+1), range(1, n+1), vtype=grb.GRB.BINARY, name="x")
        
        # Total cost initialization
        total_cost = 0
        
        # Constraints for x
        for v in range(1, n+1):
            model.addConstr(grb.quicksum(x[v, t] for t in range(1, n+1)) == 1)
            model.addConstr(grb.quicksum(x[t, v] for t in range(1, n+1)) == 1)
        
        for constraint in constraints:
            print(f"    Shop: {constraint['shop']}")
            if constraint["type"] == "lot_change":
                print("Type: lot_change")
                cl = constraint["cost"]
                k = len(constraint["partition"])
                Ul = constraint["partition"]
                print(f"UL is {Ul}")
                
                # Decision variables
                u = model.addVars(range(1, n+1), range(1, k+1), vtype=grb.GRB.BINARY, name="u")
                f = model.addVars(range(1, n+1), vtype=grb.GRB.BINARY, name="f")
                
                # Constraints for u
                for t in range(1, n+1):
                    for o in range(1, k+1):
                        model.addConstr(u[t, o] == grb.quicksum(x[v, t] for v in Ul[o-1]))
                
                # Constraints for f
                for t in range(1, n):
                    for o in range(1, k+1):
                        model.addConstr(f[t] >= u[t, o] - u[t+1, o])
                
                # Add total cost for lot_change
                total_cost += cl * grb.quicksum(f[t] for t in range(1, n+1))
            
            elif constraint["type"] == "rolling_window":
                print("Type: rolling_window")
                q = model.addVars(range(1, n+1), vtype=grb.GRB.BINARY, name="q")
                z = model.addVars(range(1, n+1), vtype=grb.GRB.INTEGER, name="z")
                model.addConstrs(z[t] >= 0 for t in range(1, n+1))  # z >= 0
                
                Vr = constraint["vehicles"]
                Mr = constraint["max_vehicles"]
                wr = constraint["window_size"]
                cr = constraint["cost"]
                
                # Constraints for q
                for t in range(1, n+1):
                    model.addConstr(q[t] <= grb.quicksum(x[v, t] for v in Vr))
                
                # Constraints for z
                for t in range(1, n+1):
                    model.addConstr(z[t] >= grb.quicksum(q[t_prime] for t_prime in range(t, min(t+wr-1, n)+1)) - Mr)
                
                # Add total cost for rolling_window
                total_cost += cr * grb.quicksum(z[t] for t in range(1, min(n-wr+1, n)+1))
            
            elif constraint["type"] == "batch_size":
                print("Type: batch_size")
                mb = constraint["min_vehicles"]
                Mb = constraint["max_vehicles"]
                Vb = constraint["vehicles"]
                cb = constraint["cost"]
                
                def gamma(t):
                    return cb * max(0, max(mb-t, t-Mb))**2
                
                p = model.addVars(range(1, n+1), vtype=grb.GRB.BINARY, name="p")
                m = model.addVars(range(1, n+1), range(1, n+1), vtype=grb.GRB.BINARY, name="m")
                l = model.addVars(range(1, n+1), range(1, n+1), vtype=grb.GRB.BINARY, name="l")
                j = model.addVars(range(1, n+1), range(1, n+1), vtype=grb.GRB.BINARY, name="j")
                g = model.addVars(range(1, n+1), range(1, n+1), vtype=grb.GRB.BINARY, name="g")
                
                # Constraints for p
                for t in range(1, n+1):
                    model.addConstr(p[t] <= grb.quicksum(x[v, t] for v in Vb))
                
                # Constraints for m, l, j
                for t in range(1, n+1):
                    for t_prime in range(t, n+1):
                        if t >= 2:
                            model.addConstr(j[t, t_prime] >= 1 - p[t])
                        for k in range(t_prime, t+1):
                            model.addConstr(m[t, t_prime] >= p[t])
                        if t_prime <= n-1:
                            model.addConstr(l[t, t_prime] >= 1 - p[t_prime])
                
                # Constraints for g
                for t in range(1, n+1):
                    for t_prime in range(t, n+1):
                        model.addConstr(g[t, t_prime] >= m[t, t_prime] + l[t, t_prime] + j[t, t_prime] - 2)
                
                # Add total cost for batch_size
                total_cost += grb.quicksum(gamma(t_prime-t+1) * g[t, t_prime] for t in range(1, n) for t_prime in range(t, n+1))
        
        # Set the objective
        model.setObjective(total_cost, grb.GRB.MINIMIZE)
        
        # Optimize the model
        model.optimize()
        
        # Output results
        if model.status == grb.GRB.OPTIMAL:
            print(f"Status: {model.status}")
            print(f"Total cost for shop {shop}: {model.objVal}")
            print("Optimized values of x:")
            x_values = []
            for v in range(1, n+1):
                for t in range(1, n+1):
                    x_values.append( x[v, t].X)
                    print(f"x[{v}, {t}] = {x[v, t].X}")
            
            # Return the values of x

            L.append(permutation(x_values,n))
            

            
        else:
            print(f"Solver termination status: {model.status}")
        print("Next shop")
    return L

In [39]:
L = solve_shop_map(shop_map,n)
print(L)


Shop: body
    Shop: body
Type: batch_size
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 55 rows, 130 columns and 180 nonzeros
Model fingerprint: 0x5eee2ca2
Variable types: 0 continuous, 130 integer (130 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Status: 2
Total cost for shop body: 0.0
Optimized values of x:
x[1, 1] = 0.0
x[1, 2] = 1.0
x[1, 